# Introducción a los Transformers y Modelos de Lenguaje de Gran Tamaño (LLMs)

En esta guía, vamos a explorar los conceptos clave detrás de los Transformers, que han revolucionado el campo del Procesamiento de Lenguaje Natural (NLP). Los modelos basados en Transformers, como BERT y GPT-3, son capaces de manejar secuencias largas de texto y capturar relaciones complejas entre palabras en diferentes posiciones del texto.

## Objetivos
- Comprender cómo funciona el mecanismo de atención en los Transformers.
- Explorar la atención multi-cabeza para captar diferentes relaciones contextuales.
- Entender cómo los Transformers incorporan la información de posición mediante codificaciones posicionales.
- Analizar la estructura de las capas de transformación y su papel en los Transformers.

## 1. Atención (Attention)

El corazón de los Transformers es el mecanismo de atención. Este mecanismo permite al modelo enfocarse en diferentes partes del texto cuando procesa una palabra, lo que es crucial para capturar dependencias a largo plazo.

La atención se calcula utilizando las matrices de Query (Q), Key (K) y Value (V):

$$
Attention(Q, K, V) = Softmax\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$

Aquí, Q representa las consultas, K las claves y V los valores. \( d_k \) es la dimensión de las claves. El producto punto entre Q y K seguido de la normalización softmax da como resultado los pesos de atención, que se usan para ponderar los valores V.

In [ ]:
# Ejemplo simple de cálculo de atención en TensorFlow
import tensorflow as tf

# Definir las matrices de Query (Q), Key (K) y Value (V)
Q = tf.random.normal(shape=(1, 5, 64))  # Query
K = tf.random.normal(shape=(1, 5, 64))  # Key
V = tf.random.normal(shape=(1, 5, 64))  # Value

# Calcular la atención
dk = tf.cast(tf.shape(K)[-1], tf.float32)
scores = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(dk)
attention_weights = tf.nn.softmax(scores, axis=-1)
attention_output = tf.matmul(attention_weights, V)
attention_output

## 2. Multi-Head Attention

Los Transformers utilizan múltiples cabezas de atención para captar diferentes tipos de relaciones contextuales en el texto. Cada cabeza procesa la información de manera diferente, y los resultados se concatenan y se combinan linealmente:

$$
MultiHead(Q, K, V) = Concat(head_1, head_2, ..., head_h)W^O
$$

Donde cada `head_i` se calcula como una instancia independiente de la atención, pero con diferentes proyecciones de Q, K y V.

In [ ]:
# Ejemplo de implementación de Multi-Head Attention en TensorFlow
multihead_attention = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=64)
output = multihead_attention(Q, K, V)
output

## 3. Codificación Posicional (Positional Encoding)

Los Transformers no procesan las secuencias en orden secuencial como las RNNs, por lo que necesitan una manera de incorporar la información de posición. Esto se hace mediante codificaciones posicionales que se suman a los embeddings de las palabras.

La codificación posicional utiliza las siguientes fórmulas para las dimensiones pares e impares del embedding:

$$
PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{model}}}\right)
$$
$$
PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{model}}}\right)
$$

In [ ]:
import numpy as np

# Implementación de codificación posicional
def positional_encoding(position, d_model):
    angle_rads = np.arange(position)[:, np.newaxis] / np.power(10000, (2 * (np.arange(d_model)[np.newaxis, :] // 2)) / d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])  # Dimensiones pares
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])  # Dimensiones impares
    return angle_rads

# Generar codificación posicional para una secuencia de 50 posiciones y un embedding de tamaño 512
pos_encoding = positional_encoding(50, 512)
pos_encoding.shape

## 4. Capas de Transformación

Los Transformers están compuestos por capas repetitivas que incluyen subcapas de atención y redes feed-forward. Cada capa sigue la estructura:

$$
LayerNorm(x + Sublayer(x))
$$

Donde `Sublayer(x)` puede ser una subcapa de atención o una red feed-forward. La normalización por capas y las conexiones residuales ayudan a estabilizar el entrenamiento y a mantener la integridad del gradiente.

In [ ]:
# Ejemplo de una capa de Transformer
from tensorflow.keras.layers import LayerNormalization, Dense, Dropout

# Definir una capa de transformación con atención y red feed-forward
class TransformerLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff):
        super(TransformerLayer, self).__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = tf.keras.Sequential([
            Dense(dff, activation='relu'),
            Dense(d_model)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(0.1)
        self.dropout2 = Dropout(0.1)

    def call(self, x, training):
        attn_output = self.mha(x, x, x)  # self-attention
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # Residual connection + normalization

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Usar la capa de Transformer
sample_transformer_layer = TransformerLayer(d_model=512, num_heads=8, dff=2048)
x = tf.random.uniform((64, 50, 512))  # Batch de secuencias
out = sample_transformer_layer(x, training=False)
out.shape